In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
import numpy as np
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

In [ ]:
df_train  = pd.read_json('/content/train.jsonl', lines=True)

In [ ]:
df_val = pd.read_json('/content/validation.jsonl', lines=True)

In [ ]:
df_test = pd.read_json('/content/test.jsonl', lines=True)

In [ ]:
df_train

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[Passion is overrated — 7 work habits you need...,Twitter,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",[phrase]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,92578045-699f-4957-a3c5-cff2c3874dae,591979258442027008,[Has Facebook's video explosion completely sha...,Twitter,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[http://djcs-prod.s3.amazonaws.com/wsjfrontpag...,http://on.wsj.com/1GfyUqz,"{'source': 'anonymized', 'humanSpoiler': 'No.'...",[it hasn’t necessarily taken the wind out of Y...,"[[[7, 50], [7, 118]]]",[passage]
3196,51682121-df0b-4289-a95f-e1bc3d181306,881531941974661_902284433232745,[Cop Is Eating At A Chili's When Teen Hands Hi...,Facebook,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,[https://sbly-web-prod-shareably.netdna-ssl.co...,http://shareably.net/officer-receives-touching...,"{'source': 'anonymized', 'humanSpoiler': 'The ...","[It read, ""Thanks for keeping us safe.""]","[[[0, 317], [0, 355]]]",[passage]
3197,9c45ca67-38c4-47b4-aa0d-48434bae09fc,837356193576333314,[5 popular myths about visible signs of aging ...,Twitter,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,Dozens of anti-wrinkle products and creams are...,,None,https://www.businessinsider.in/5-popular-myths...,"{'source': 'anonymized', 'humanSpoiler': None,...",[1. Anti-wrinkle creams will er

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'phrase':0,
          'passage':1,
          'multi':2,
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label[0]] for label in df['tags']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['postText']]

    def classes(self):
        return self.labels

    def __len__(self): 
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class TestDataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['postText']]

    def classes(self):
        return self.texts

    def __len__(self): 
        return len(self.texts)
        
    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        return batch_texts

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.1):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 3)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .2f} | Train Accuracy: {total_acc_train / len(train_data): .2f} | Val Loss: {total_loss_val / len(val_data): .2f} | Val Accuracy: {total_acc_val / len(val_data): .2f}')
                 

In [ ]:
def evaluate(model, test_data):

    test = TestDataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input in test_dataloader:

              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              pred=output.argmax(dim=1)
              print(pred.item())


In [ ]:
print(len(df_train),len(df_val), len(df_test))
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)


In [ ]:
test_array=np.array(test_array,dtype=int)

In [ ]:
test_array=evaluate(model, df_test)

0
0
0
0
0
1
1
1
0
2
1
1
0
2
0
1
2
1
1
0
0
0
1
0
0
0
1
2
1
1
1
2
1
0
1
1
1
1
0
0
0
2
0
0
1
2
1
1
0
0
0
0
1
2
0
0
0
2
1
1
0
1
0
1
1
1
0
1
1
0
2
1
1
0
1
1
1
1
0
1
1
1
2
0
0
0
2
1
0
1
1
1
0
1
0
0
0
1
1
0
2
1
0
0
1
0
0
0
0
2
2
0
0
0
0
0
0
1
1
0
0
1
0
1
0
0
0
1
0
0
0
0
2
1
0
0
0
2
1
0
0
0
2
1
1
1
0
1
2
0
0
0
1
0
0
1
0
0
1
1
0
0
1
1
0
0
1
0
1
0
0
0
0
0
1
1
0
0
0
0
0
2
0
0
0
0
0
1
0
0
0
1
0
1
1
0
2
0
1
1
1
1
0
2
1
0
0
0
0
0
0
0
2
0
1
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
2
0
1
0
2
1
0
0
0
1
0
0
0
0
1
0
0
0
1
2
0
0
1
2
2
0
0
1
1
1
0
1
0
0
2
0
1
0
0
0
1
0
1
0
1
0
1
2
0
1
0
0
0
0
1
1
0
1
0
0
0
1
0
0
1
1
1
1
0
1
0
0
2
0
2
1
1
0
0
1
0
1
0
0
0
1
1
0
0
1
0
0
0
0
0
2
0
0
1
1
0
1
1
0
0
0
0
0
0
2
0
0
0
0
0
0
0
1
1
0
1
1
0
0
0
0
1
1
1
1
2
1
1
1
0
0
0
2
1
0
0
0
2
0
0
0
2
1
0
1
0
0
1
0
2
0
0
0
0
0
1
1
1
0
1
1
1
0
0
1
0
0
1
0
1
2
1
1
1
1
0
0
1
0
1
0
2
2
0
1
0
1
1
0
0
0
0
0
0
0
0
0
1
0
1
2
1
0
0
2
0
1
1
1
0
1
0
0
2
0
1
0
1
2
0
2
0
0
0
0
2
0
0
0
0
0
0
1
1
1
0
0
1
0
1
0
2
1
1
1
1
1
0
0
1
1
0
0
0
1


In [ ]:
len(test_array)

1000

In [ ]:
predictions=[]
for i in test_array:
    if i == 0:
        predictions.append("phrase")
    elif i == 1:
        predictions.append("passage")
    elif i == 2:
        predictions.append("multi")
    else:
        print(i)
        predictions.append("other")

In [ ]:
predlist=[]

for i in range(len(df_test)):
    preds={}
    preds["uuid"]=df_test["uuid"][i]
    preds["spoilerType"]=predictions[i]
    predlist.append(preds)

In [ ]:
predlist

[{'uuid': '2f3e30d7-972c-4812-b727-6b582de52137', 'spoilerType': 'phrase'},
 {'uuid': '45425f74-4c0e-420c-aea8-6337ea91e107', 'spoilerType': 'phrase'},
 {'uuid': '2fb84020-cfdb-4b7f-9cfc-b891c306c946', 'spoilerType': 'phrase'},
 {'uuid': '7cc8b816-a3fe-4b3f-9568-b353f87a3161', 'spoilerType': 'phrase'},
 {'uuid': 'b90a7343-ab27-4750-8e12-43f3f03fc9e5', 'spoilerType': 'phrase'},
 {'uuid': '1b83a7b8-72fa-4b21-8dd3-a5cff484d791', 'spoilerType': 'passage'},
 {'uuid': 'b63a63ce-38be-498d-8eca-f571d713e05c', 'spoilerType': 'passage'},
 {'uuid': 'fd5ef1e7-63ce-499c-9fc3-00d5209ccef6', 'spoilerType': 'passage'},
 {'uuid': '3802f457-cee5-4324-80a0-161cc387785b', 'spoilerType': 'phrase'},
 {'uuid': '2be7bbfb-5c97-4b1e-925d-8d76fff2d9da', 'spoilerType': 'multi'},
 {'uuid': '92de48d8-4b48-40b9-a0dd-83c28fb3da58', 'spoilerType': 'passage'},
 {'uuid': '1de9243c-3882-42f1-84ef-21c3b4176f36', 'spoilerType': 'passage'},
 {'uuid': 'c11945bb-a85e-4859-a0e0-0234eb264a91', 'spoilerType': 'phrase'},
 {'uuid'